In [1]:
import os
import google.generativeai as genai
import json
from dotenv import load_dotenv

An error occurred: module 'importlib.metadata' has no attribute 'packages_distributions'


/Users/nissiotoo/Library/Python/3.9/lib/python/site-packages/google/api_core/_python_version_support.py:252: FutureWarning: You are using a Python version (3.9.6) past its end of life. Google will update google.api_core with critical bug fixes on a best-effort basis, but not with any other fixes or features. Please upgrade to the latest Python version, or at least Python 3.10, and then update google.api_core.
  warnings.warn(message, FutureWarning)
/Users/nissiotoo/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/nissiotoo/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load environment variables .env
load_dotenv()

# Get the Gemini API key from .env
api_key = os.getenv('GEMINI_API_KEY')

In [ ]:
# Configure Gemini with API key
genai.configure(api_key=api_key)

# Debug: Print available models
print("Available models:", [m.name for m in genai.list_models()])

# Create the model
model = genai.GenerativeModel('models/gemini-pro-latest')


Available models: ['models/embedding-gecko-001', 'models/gemini-2.5-pro-preview-03-25', 'models/gemini-2.5-flash-preview-05-20', 'models/gemini-2.5-flash', 'models/gemini-2.5-flash-lite-preview-06-17', 'models/gemini-2.5-pro-preview-05-06', 'models/gemini-2.5-pro-preview-06-05', 'models/gemini-2.5-pro', 'models/gemini-2.0-flash-exp', 'models/gemini-2.0-flash', 'models/gemini-2.0-flash-001', 'models/gemini-2.0-flash-exp-image-generation', 'models/gemini-2.0-flash-lite-001', 'models/gemini-2.0-flash-lite', 'models/gemini-2.0-flash-lite-preview-02-05', 'models/gemini-2.0-flash-lite-preview', 'models/gemini-2.0-pro-exp', 'models/gemini-2.0-pro-exp-02-05', 'models/gemini-exp-1206', 'models/gemini-2.0-flash-thinking-exp-01-21', 'models/gemini-2.0-flash-thinking-exp', 'models/gemini-2.0-flash-thinking-exp-1219', 'models/gemini-2.5-flash-preview-tts', 'models/gemini-2.5-pro-preview-tts', 'models/learnlm-2.0-flash-experimental', 'models/gemma-3-1b-it', 'models/gemma-3-4b-it', 'models/gemma-3-12

In [ ]:
class AccountSummaryAgent:
    def __init__(self, model_name="gemini-2.5-flash"):
        self.model = genai.GenerativeModel(model_name)
 
    def create_summary(self, account_data: dict, health_score: float = None) -> str:
        """
        Generate an executive-level account summary from account data.
        
        Args:
            account_data (dict): Account information including:
                - account_name: Name of the account
                - industry/sector: Industry vertical
                - annual_revenue: Annual revenue
                - recent_activity: List of recent activities
                - open_opportunities: List of open opportunities
                - risk_factors: List of identified risks
                - Any other relevant account details
            health_score (float): Optional health score (0-1 scale)
            
        Returns:
            str: Generated account summary with analysis and recommendations
        """
        
        # ✅ FIX: Format health score BEFORE using it in the f-string
        if health_score is not None:
            health_score_str = f"{health_score:.2f}"
        else:
            health_score_str = "Not available"
        
        prompt = f"""
You are an expert CRM account analyst.

You will be given structured account data and a predictive health score.
Return a concise, executive-level account summary.

Account Health Score: {health_score_str}

Account Data (JSON):
{json.dumps(account_data, indent=2)}

Generate the following sections:

1. **Executive Summary** (3-4 sentences)
2. **Key Strengths** (bullet points)
3. **Key Risks or Warning Signs** (bullet points)
4. **Open Opportunities** (bullet points if any)
5. **Recommended Next Actions** (3 concrete steps with owners + timelines)
6. **Overall Priority Level** (High / Medium / Low)

Be specific, data-driven, and professional.
"""
        
        response = self.model.generate_content(prompt)
        return response.text

In [5]:
# ------- TEST THE ACCOUNT SUMMARY AGENT -------
 
# Sample account data for testing
test_account_data = {
    "account_name": "Acme Corp",
    "industry": "FinTech",
    "annual_revenue": 8500000,
    "recent_activity": ["QBR scheduled", "Support ticket escalated"],
    "open_opportunities": ["Renewal expansion", "Cross-sell analytics package"],
    "risk_factors": ["Reduced logins", "Competitor outreach noticed"]
}
 
# Create an instance of the agent
agent = AccountSummaryAgent(model_name="gemini-2.5-flash")
 
# Generate the summary
output = agent.create_summary(test_account_data, health_score=0.72)
 
# Print the result
print("=== ACCOUNT SUMMARY ===\n")
print(output)
print("\n========================")
 

=== ACCOUNT SUMMARY ===

**Executive-Level Account Summary: Acme Corp**

**1. Executive Summary**
Acme Corp, a significant FinTech client with $8.5M in annual revenue, currently holds a health score of 0.72. While there are active engagement points like a scheduled QBR and two promising growth opportunities (renewal expansion, cross-sell analytics), the account exhibits concerning risk factors. Reduced product logins, noted competitor outreach, and an escalated support ticket collectively indicate potential disengagement and dissatisfaction that require immediate intervention to secure the relationship and capitalize on growth.

**2. Key Strengths**
*   Established FinTech client with substantial annual revenue ($8.5M).
*   Active engagement with an upcoming Quarterly Business Review (QBR).
*   Two clear open opportunities for revenue growth: renewal expansion and a cross-sell of an analytics package.

**3. Key Risks or Warning Signs**
*   **Reduced Product Logins:** Indicates potentia